In [ ]:
from pandas_datareader import data as wb
import os
from stocks import stocks

for s in stocks:
    d = wb.DataReader(s, data_source='yahoo', start='1997-1-1')
    file_path = os.path.join('data', s+'.csv')
    d.to_csv(file_path)

In [12]:
%load_ext autoreload
%autoreload 2

# Manual input
usd_to_sgd_rate = 1.3304      # USD -> SGD
remaining_usd_cash = 1371.55  # Remaining USD cash in IB and DBS

def load_data(tick):
    file_path = os.path.join('data', tick+'.csv')
    data = pd.read_csv(file_path)
    dt_idx = pd.DatetimeIndex(data['Date'])
    data.index = dt_idx

    return data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import pandas as pd
import os

stock_buy_transactions = pd.read_csv('stock_buy_transactions.csv')
stock_dividends = pd.read_csv('stock_dividends.csv')
stocks = pd.read_csv('stocks.csv').fillna(0)
stocks = stocks.set_index('Unnamed: 0')
df = pd.DataFrame()
df['tick'] = stock_buy_transactions['tick'].drop_duplicates()

df = pd.merge(df, stock_dividends.groupby(['tick']).sum(), on='tick', how='outer')
df = df.rename(columns={'amount':'dividends'})
df['dividends'] = df['dividends'].fillna(0)
df = df.drop(columns=['Unnamed: 0'])

stock_buy_transactions['total_cost'] = stock_buy_transactions['buy_price']*stock_buy_transactions['quantity']\
    +stock_buy_transactions['commission']
df = pd.merge(df, stock_buy_transactions.groupby(['tick']).sum(), on='tick', how='outer')
df['avg_cost'] = df['total_cost']/df['quantity']
df = df.drop(columns=['Unnamed: 0', 'buy_price'])

df['currency_sgd'] = 'SGD'
df['current_price'] = 0.0
for index, row in df.iterrows():
    t = row['tick']
    d = load_data(t)
    current_price = d['Adj Close'].iloc[-1]
    df.at[index, 'current_price'] = current_price
    df.at[index, 'currency_sgd'] = 1
    if stocks[t]['currency']=='USD':
        df.at[index, 'currency_sgd'] = usd_to_sgd_rate

df['pl'] = df['current_price']*df['quantity']+df['dividends']-df['total_cost']
df['pl_percent'] = (df['current_price']*df['quantity']+df['dividends'])/df['total_cost']-1.0
df['current_value_sgd'] = df['current_price']*df['quantity']*df['currency_sgd']
df['allocation'] = df['current_value_sgd']/df['current_value_sgd'].sum()
df

,tick,dividends,quantity,commission,total_cost,avg_cost,currency_sgd,current_price,pl,pl_percent,current_value_sgd,allocation
0,D05.SI,126.00,700,24.61,14304.610,20.435157,1,29.910000,6758.389893,0.472462,20937,0.0649907
1,ME8U.SI,286.80,5000,33.18,15253.180,3.050636,1,2.830000,-816.380381,-0.053522,14150,0.0439231
2,AJBU.SI,134.26,3500,16.70,10026.700,2.864771,1,2.690000,-477.439800,-0.047617,9415,0.0292252
3,C2PU.SI,142.20,2000,14.25,8334.250,4.167125,1,4.170000,147.950153,0.017752,8340,0.0258883
4,BABA,-1.87,91,77.04,22396.824,246.118945,1.3304,230.949997,-1382.244278,-0.061716,27960.3,0.0867918
5,AAPL,9.24,62,19.26,7081.680,114.220645,1.3304,131.460007,1078.080416,0.152235,10843.5,0.0336593
6,MSFT,9.05,34,19.26,7101.800,208.876471,1.3304,252.179993,1481.369751,0.208591,11407,0.0354086
7,NVDA,0.00,18,19.26,9636.120,535.340000,1.3304,600.380005,1170.720088,0.121493,14377.4,0.0446291
8,ARKK,108.14,95,20.26,9869.460,103.889053,1.3304,120.769997,1711.829681,0.173447,15263.9,0.0473807
9,SPY,20.05,22,19.26,7607.280,345.785455,1.3304,417.299988,1593.369731,0.209453,12213.9,0.0379132


In [14]:
stock_others = pd.read_csv('stock_others.csv') # TODO: not calculated yet

print('SG stocks PL:')
print(df[df['currency_sgd']==1]['pl'].sum())
print('\nUS stocks PL (in SGD):')
print(df[df['currency_sgd']!=1]['pl'].sum()*usd_to_sgd_rate)
print('\nTotal PL (in SGD)')
print(df[df['currency_sgd']==1]['pl'].sum()+df[df['currency_sgd']!=1]['pl'].sum()*usd_to_sgd_rate)

SG stocks PL:
5612.519864578246

US stocks PL (in SGD):
11024.92493628159

Total PL (in SGD)
16637.444800859834


In [21]:
from datetime import datetime, timedelta

currency_exchange_transactions = pd.read_csv('currency_exchange_transactions.csv')
currency_exchange_transactions = currency_exchange_transactions.set_index('Unnamed: 0')
total_original_sgd_to_usd = currency_exchange_transactions['amount'].sum()
print('Total Converted SGD to USD\t' + str(total_original_sgd_to_usd))

total_sgd_cost = df[df['currency_sgd']==1]['total_cost'].sum()
print('Total SGD Stocks Cost\t\t' + str(total_sgd_cost) + '\n')

usd_total_remain_sgd = df[df['currency_sgd']!=1]['current_value_sgd'].sum()+remaining_usd_cash*usd_to_sgd_rate
sgd_pl = df[df['currency_sgd']==1]['pl'].sum()
now = datetime.now()
stock_buy_transactions['expense_ratio']=0.0
stock_buy_transactions['days'] = stock_buy_transactions['time'].apply(\
                                                            lambda x: (now-datetime.strptime(x, '%Y-%m-%d')).days)
for index, row in stock_buy_transactions.iterrows():
    stock_buy_transactions.at[index, 'expense_ratio'] = stocks[row['tick']]['expense_ratio']
stock_buy_transactions['expense']=stock_buy_transactions['days']/365*stock_buy_transactions['expense_ratio']\
                                    *stock_buy_transactions['total_cost']/100*usd_to_sgd_rate
total_expense = stock_buy_transactions['expense'].sum()
actual_pl = usd_total_remain_sgd-total_original_sgd_to_usd+sgd_pl-total_expense

print('Actual Total PL (in SGD):\t' + str(actual_pl))
print('Actual Total PL%\t\t' + '{:.2%}'.format(actual_pl/(total_original_sgd_to_usd+total_sgd_cost)))

stock_buy_transactions.loc[(stock_buy_transactions.currency == 'SGD'),'currency']=1.0
stock_buy_transactions.loc[(stock_buy_transactions.currency == 'USD'),'currency']=usd_to_sgd_rate
stock_buy_transactions['annualized_cost_sgd'] = stock_buy_transactions['currency']*stock_buy_transactions['days']\
                                                /365*stock_buy_transactions['total_cost']
annualized_cost_sgd = stock_buy_transactions['annualized_cost_sgd'].sum()
print(annualized_cost_sgd/(total_original_sgd_to_usd+total_sgd_cost))
print(actual_pl/annualized_cost_sgd)

Total Converted SGD to USD	263472.44175
Total SGD Stocks Cost		47918.740000000005

Actual Total PL (in SGD):	13012.094961182724
Actual Total PL%		4.18%
0.3579838550098472
0.11672865297727794


In [41]:
transfer_to_ib = []
transfer_to_ib.append({'time':'2021-3-2', 'currency':'USD', 'amount':7571})
transfer_to_ib.append({'time':'2021-3-2', 'currency':'SGD', 'amount':14831})
transfer_to_ib.append({'time':'2021-3-4', 'currency':'SGD', 'amount':25000})
transfer_to_ib.append({'time':'2021-3-8', 'currency':'SGD', 'amount':2470})
transfer_to_ib.append({'time':'2021-4-16', 'currency':'SGD', 'amount':17218})
transfer_to_ib.append({'time':'2021-4-23', 'currency':'SGD', 'amount':30000})
transfer_to_ib.append({'time':'2021-4-23', 'currency':'SGD', 'amount':2000})

#d = pd.DataFrame(transfer_to_ib)
#d.to_csv('transfer_to_ib_transactions.csv')

transfer_to_ib_transactions = pd.read_csv('transfer_to_ib_transactions.csv')
transfer_to_ib_transactions

total_transfered_sgd = transfer_to_ib_transactions[transfer_to_ib_transactions.currency=='SGD']['amount'].sum()

currency_exchange_transactions['coverted_amount'] = currency_exchange_transactions['amount']/\
                            currency_exchange_transactions['rate']
converted_sgd = currency_exchange_transactions[currency_exchange_transactions.place=='ib']\
                            [currency_exchange_transactions.currency=='SGDUSD']['amount'].sum()
print(converted_sgd)
converted_usd = currency_exchange_transactions[currency_exchange_transactions.place=='ib']\
                            [currency_exchange_transactions.currency=='SGDUSD']['coverted_amount'].sum()
print(converted_usd)
stock_sgd_purchase = stock_buy_transactions[stock_buy_transactions.currency==1]\
    [stock_buy_transactions['commission']<5]['total_cost'].sum()
print(stock_sgd_purchase)
currency_exchange_transactions
stock_buy_transactions

print(total_transfered_sgd-converted_sgd-stock_sgd_purchase) # this number should be very close

# in USD
print(converted_usd+stock_sgd_purchase/current_rate)

89681.44175000001
67399.52921049757
1836.5
1.0582499999873107


<ipython-input-41-a6ff13ed72f1>:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  converted_sgd = currency_exchange_transactions[currency_exchange_transactions.place=='ib']\
<ipython-input-41-a6ff13ed72f1>:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  converted_usd = currency_exchange_transactions[currency_exchange_transactions.place=='ib']\
<ipython-input-41-a6ff13ed72f1>:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stock_sgd_purchase = stock_buy_transactions[stock_buy_transactions.currency==1]\


NameError: name 'current_rate' is not defined

In [22]:
# calculate ib account
transfer_to_ib = []
transfer_to_ib.append({'time':'2021-3-2', 'currency':'USD', 'amount':7571})
transfer_to_ib.append({'time':'2021-3-2', 'currency':'SGD', 'amount':14831})
transfer_to_ib.append({'time':'2021-3-4', 'currency':'SGD', 'amount':25000})
transfer_to_ib.append({'time':'2021-3-8', 'currency':'SGD', 'amount':2470})
transfer_to_ib.append({'time':'2021-4-16', 'currency':'SGD', 'amount':17218})
transfer_to_ib.append({'time':'2021-4-23', 'currency':'SGD', 'amount':30000})
transfer_to_ib.append({'time':'2021-4-23', 'currency':'SGD', 'amount':2000})

total_transfered_sgd = 0
for item in transfer_to_ib:
    if item['currency'] == 'SGD':
        total_transfered_sgd += item['amount']
print(total_transfered_sgd)

converted_sgd = 0
converted_usd = 0
for item in currency_exchange_transactions:
    if item['place'] == 'ib' and item['currency'] == 'SGDUSD':
        converted_sgd += item['amount']
        converted_usd += item['amount']/item['rate']
        
stock_sgd_purchase = 0      

for item in transactions:
    if item['currency'] == 'SGD' and item['commission'] < 5:
        stock_sgd_purchase = stock_sgd_purchase + item['buy_price']*item['quantity'] + item['commission']

print(converted_sgd)
print(total_transfered_sgd-converted_sgd-stock_sgd_purchase) # this number should be very close

# in USD
print(converted_usd+stock_sgd_purchase/current_rate)

91519


TypeError: string indices must be integers

In [ ]:
# Simulation Need to reimplment
from common import get_price

monthly_new_invest = 15000
start_td = datetime.strptime('2007-3-1', '%Y-%m-%d')

def simulate_stock(tick, start_td):
    print(tick)
    end_td = datetime.now()
    month_cnt = 0
    td = start_td
    d = load_data(tick)
    first_td = start_td = datetime.strptime(d['Date'].iloc[0], '%Y-%m-%d')
    if first_td > td:
        td = first_td

    simulated_transactions = []

    while td < end_td:
        price = get_price(d, td.strftime('%Y-%m-%d'))
        if not price:
            break
        simulated_transactions.append({'date': td, 'price': price, 'quantity': round(monthly_new_invest/price)})
        month_cnt += 1
        td += timedelta(days=30.4)

    total_money_for_invest = monthly_new_invest * month_cnt
    print(total_money_for_invest)

    market_value = 0
    total_quantity = 0
    total_cost = 0
    for st in simulated_transactions:
        total_quantity += st['quantity']
        total_cost += st['quantity']*st['price']
    market_value = total_quantity * d['Adj Close'].iloc[-1]
    print(total_cost)
    print(market_value)
    print(market_value/total_cost-1)
    print((market_value/total_cost-1)/(month_cnt/12))

for s in splits['USD']:   
    simulate_stock(s, start_td)